### dev_notebook_

This is your development environment...you may start:
- Exploring your data.
- Testing the ad-hoc methods.
- Designing your pipeline.

In [301]:
import pandas as pd 
import requests as req




#### **Bicimad**

In [302]:
df_csv = pd.read_csv('../data/bicimad_stations.csv', sep='\t') #importamos el csv

df_csv


,Unnamed: 0,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates
0,0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]"
1,1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]"
2,2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]"
3,3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]"
4,4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,259,265,257 - INEF,0,257,Avenida Juan de Herrera frente a la calle Paul...,1,0,24,3,19,0,Point,"[-3.72997, 40.43896]"
260,260,266,258 - Ciudad Universitaria 1,2,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,9,14,0,Point,"[-3.72699, 40.44375]"
261,261,267,259 - Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,10,0,Point,"[-3.72693, 40.44342]"
262,262,268,260 - Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,12,10,0,Point,"[-3.7272945, 40.4483322]"


In [303]:
df_csv.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            264 non-null    int64 
 1   id                    264 non-null    int64 
 2   name                  264 non-null    object
 3   light                 264 non-null    int64 
 4   number                264 non-null    object
 5   address               264 non-null    object
 6   activate              264 non-null    int64 
 7   no_available          264 non-null    int64 
 8   total_bases           264 non-null    int64 
 9   dock_bikes            264 non-null    int64 
 10  free_bases            264 non-null    int64 
 11  reservations_count    264 non-null    int64 
 12  geometry.type         264 non-null    object
 13  geometry.coordinates  264 non-null    object
dtypes: int64(9), object(5)
memory usage: 29.0+ KB


In [304]:
df_csv['geometry.type'].value_counts() 


geometry.type
Point    264
Name: count, dtype: int64

In [305]:
#elimino dos columnas que no aportan valor
bicimad=df_csv.drop(columns='geometry.type') 
bicimad=bicimad.drop(columns='Unnamed: 0') 
bicimad

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.coordinates
0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,"[-3.7018341, 40.4172137]"
1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,"[-3.701602938060457, 40.41731271011562]"
2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,"[-3.7058415, 40.4205886]"
3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,"[-3.7069171, 40.4302937]"
4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,"[-3.7025875, 40.4285524]"
...,...,...,...,...,...,...,...,...,...,...,...,...
259,265,257 - INEF,0,257,Avenida Juan de Herrera frente a la calle Paul...,1,0,24,3,19,0,"[-3.72997, 40.43896]"
260,266,258 - Ciudad Universitaria 1,2,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,9,14,0,"[-3.72699, 40.44375]"
261,267,259 - Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,10,0,"[-3.72693, 40.44342]"
262,268,260 - Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,12,10,0,"[-3.7272945, 40.4483322]"


In [306]:
#compruebo los NaN
nan_cols = bicimad.isna().sum() 
nan_cols[nan_cols>0]

Series([], dtype: int64)

#### **Bicipark**


In [307]:
bicipark = pd.read_csv('../data/bicipark_stations.csv', sep=';') #importamos el csv
bicipark.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            11 non-null     int64 
 1   stationId             11 non-null     int64 
 2   stationName           11 non-null     object
 3   address               11 non-null     object
 4   city                  11 non-null     object
 5   zip_code              11 non-null     object
 6   total_places          11 non-null     int64 
 7   free_places           11 non-null     int64 
 8   enabled               11 non-null     int64 
 9   reserved_places       11 non-null     int64 
 10  geometry.type         11 non-null     object
 11  geometry.coordinates  11 non-null     object
dtypes: int64(6), object(6)
memory usage: 1.2+ KB


In [308]:
bicipark.head()

,Unnamed: 0,stationId,stationName,address,city,zip_code,total_places,free_places,enabled,reserved_places,geometry.type,geometry.coordinates
0,0,74,Bicipark Fuente de la Mora,Estación de Fuente de la Mora,Madrid,28050,10,9,1,0,Point,"[-3.6630679, 40.4846838]"
1,1,22,Bicipark Orense,"Avenida General Perón, 27",Madrid,28020,15,9,1,1,Point,"[-3.693897, 40.452973]"
2,2,49,Bicipark Montalban,"Calle de Montalbán, 5",Madrid,28014,16,15,1,0,Point,"[-3.690786, 40.418146]"
3,3,12,Bicipark Almagro,"Calle Almagro, 11",Madrid,28010,16,12,1,0,Point,"[-3.692879, 40.430361]"
4,4,9,Bicipark Recoletos,"Paseo de Recoletos, 2",Madrid,28001,10,3,0,0,Point,"[-3.692245, 40.420354]"


In [309]:
#compruebo los NaN
nan_cols = bicipark.isna().sum() 
nan_cols[nan_cols>0]

Series([], dtype: int64)

#### **API**

In [310]:
url = 'https://datos.madrid.es/egob'
veronica='/catalogo/201000-0-embajadas-consulados.json'
url=url + veronica

res = req.get(url)  
res


<Response [200]>

In [311]:
data=res.json()
#data

In [312]:
data.keys()

dict_keys(['@context', '@graph'])

In [313]:
data['@context'].keys()

dict_keys(['c', 'dcterms', 'geo', 'loc', 'org', 'vcard', 'schema', 'title', 'id', 'relation', 'references', 'address', 'area', 'district', 'locality', 'postal-code', 'street-address', 'location', 'latitude', 'longitude', 'organization', 'organization-desc', 'accesibility', 'services', 'schedule', 'organization-name', 'description', 'link', 'uid', 'dtstart', 'dtend', 'time', 'excluded-days', 'event-location', 'free', 'price', 'recurrence', 'days', 'frequency', 'interval', 'audience'])

In [314]:
data['@context']['geo']

'http://www.w3.org/2003/01/geo/wgs84_pos#'

In [315]:
data['@graph'][1]['organization']['organization-desc'].split('\xa0\xa0')[1].split('.  ')[:3]

['Metro: Concha Espina (línea 9), Colombia (líneas 8 y 9)',
 'Bus: 51, 7, 120, 43',
 'Bicimad: Estaciones 148 (calle Serrano, 210), 159 (paseo de la Habana, 63).']

In [317]:
graph=data['@graph']
graph[0]['organization']['organization-desc'].split('\xa0\xa0')[1].split('.  ')



['Metro: Alonso Martínez (líneas 4, 5 y 10)',
 'Bus: 7, 21, C03, 37, 3, 14, 150, 27, 45, 5',
 'Bicimad: Estaciones 8 (plaza de Alonso Martínez, 5), 10 (calle marqués de la Ensenada, 16), 104 (paseo de la Castellana, 4)',
 'Aparcamiento: Almagro (12) calle Almagro 11; Villa de París (67), plaza Villa de París.']

In [318]:
def encontrar_elemento_con_palabra(lista, palabra):
    for elemento in lista:
        if palabra in elemento:
            return elemento
        else:
            pass
            
    return 'No tiene Bicimad'
    
   

In [339]:
org=[]
tem_list=[]    
for dic in graph:
    listas=dic['organization']['organization-desc'].split('.  ') 
    tem_list.append(listas)
for lista in tem_list:
    e=encontrar_elemento_con_palabra(lista,'Bicimad')
    org.append(e)
           
#org


In [334]:
info=data['@graph']

In [203]:
dp=pd.DataFrame(info)
dp.head()


,@id,@type,id,title,relation,address,location,organization
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,7933,Consulado de Argentina,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.42821624776458, 'longitude': -...",{'organization-desc': ' Metro: Alonso Martíne...
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,10197947,Consulado de Belice,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.453375787625184, 'longitude': ...",{'organization-desc': ' Metro: Concha Espina ...
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,21112,Consulado de Bolivia,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.44244658703542, 'longitude': -...",{'organization-desc': ' Metro: Cruz del Rayo ...
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,64260,Consulado de Brasil,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.42571641206772, 'longitude': -...",{'organization-desc': ' Metro: Serrano (línea...
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,10197938,Consulado de Bélgica,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.428625519984514, 'longitude': ...",{'organization-desc': ' Metro: Serrano (línea...


In [93]:
org= pd.DataFrame(pd.json_normalize(dp['organization']))
org.head()

,organization-desc,accesibility,schedule,services,organization-name
0,"Metro: Alonso Martínez (líneas 4, 5 y 10). ...",3,,,Consulado de Argentina
1,"Metro: Concha Espina (línea 9), Colombia (lí...",3,,,Consulado de Belice
2,"Metro: Cruz del Rayo (línea 9), Prosperidad ...",3,,,Consulado de Bolivia
3,"Metro: Serrano (línea 4). Bus: 21, 53, 1, 1...",3,,,Consulado de Brasil
4,"Metro: Serrano (línea 4), Colón (línea 4), R...",3,,,Consulado de Bélgica


In [92]:
adress= pd.DataFrame(pd.json_normalize(dp['address']))
adress.head()

,locality,postal-code,street-address,district.@id,area.@id
0,MADRID,28010,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...
1,MADRID,28016,CALLE TALAVERA 9,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...
2,MADRID,28002,CALLE AVIADOR LINDBERGH 3,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...
3,MADRID,28001,CALLE GOYA 5 y 7 pasaje,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...
4,MADRID,28046,PASEO CASTELLANA 18 PLANTA 6 Izquierda,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...


In [344]:
concatenated_df=pd.concat([dp, adress], axis=1).drop(columns=['id','@id','@type','relation','address', 'organization','district.@id','area.@id'])
concatenated_df['bicimad']=org
concatenated_df['type_of_place']='Embajadas y Consulados'
concatenated_df.head()

,title,location,locality,postal-code,street-address,bicimad,type_of_place
0,Consulado de Argentina,"{'latitude': 40.42821624776458, 'longitude': -...",MADRID,28010,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,Bicimad: Estaciones 8 (plaza de Alonso Martíne...,Embajadas y Consulados
1,Consulado de Belice,"{'latitude': 40.453375787625184, 'longitude': ...",MADRID,28016,CALLE TALAVERA 9,"Bicimad: Estaciones 148 (calle Serrano, 210), ...",Embajadas y Consulados
2,Consulado de Bolivia,"{'latitude': 40.44244658703542, 'longitude': -...",MADRID,28002,CALLE AVIADOR LINDBERGH 3,"Bicimad: Estación 146 (calle María Francisca, 1).",Embajadas y Consulados
3,Consulado de Brasil,"{'latitude': 40.42571641206772, 'longitude': -...",MADRID,28001,CALLE GOYA 5 y 7 pasaje,"Bicimad: Estaciones 93 (calle Goya, 1) y 106 a...",Embajadas y Consulados
4,Consulado de Bélgica,"{'latitude': 40.428625519984514, 'longitude': ...",MADRID,28046,PASEO CASTELLANA 18 PLANTA 6 Izquierda,Bicimad: Estaciones 104 (paseo de la Castellan...,Embajadas y Consulados


In [355]:
concatenated_df.loc[0,'bicimad'].split('Bicimad: Estaciones ')[1]


'8 (plaza de Alonso Martínez, 5), 10 (calle marqués de la Ensenada, 16), 104 (paseo de la Castellana, 4)'